# SQL Analysis for Telecom Business Insights
## Notebook 08: Data Analyst Portfolio Showcase

This notebook demonstrates **Advanced SQL** capabilities by querying the project's database directly. 
While previous notebooks used Python/Pandas, this section focuses on the core skill set required for Data Analyst roles: **extracting actionable insights from relational databases**.

### Key Skills Demonstrated:
- Complex Joins & Aggregations
- Window Functions (Rank, Partition By)
- CTEs (Common Table Expressions)
- Case Statements & Business Logic
- Cohort Analysis

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to the SQLite database created by src/create_db.py
conn = sqlite3.connect('../data/telecom.db')

def run_query(query):
    """Execute SQL query and return DataFrame."""
    return pd.read_sql(query, conn)

# List all tables
tables = run_query("SELECT name FROM sqlite_master WHERE type='table';")
print("Available Tables:")
print(tables)

## 1. High-Value Customer Segmentation
**Business Question:** Who are our top revenue-generating customers, and where do they live?
**SQL Technique:** `JOIN`, `GROUP BY`, `ORDER BY`

In [ ]:
query_1 = """
SELECT 
    d.city,
    COUNT(d.customer_id) as customer_count,
    ROUND(AVG(u.arpu), 2) as avg_arpu,
    SUM(CASE WHEN u.arpu > 75 THEN 1 ELSE 0 END) as high_value_customers,
    ROUND(SUM(u.arpu), 2) as total_monthly_revenue
FROM demographics d
JOIN usage_billing u ON d.customer_id = u.customer_id
GROUP BY d.city
ORDER BY total_monthly_revenue DESC
LIMIT 10;
"""

df_1 = run_query(query_1)
display(df_1)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(data=df_1, x='avg_arpu', y='city', palette='viridis')
plt.title('Average ARPU by City (SQL Output)')
plt.xlabel('Average Monthly Revenue ($)')
plt.show()

## 2. Churn Risk Analysis using Window Functions
**Business Question:** Identify the top 3 customers with the highest number of complaints in each city.
**SQL Technique:** `WINDOW FUNCTION (RANK)`, `CTE`

In [ ]:
query_2 = """
WITH CityRankings AS (
    SELECT 
        d.customer_id,
        d.city,
        c.num_complaints_12m,
        u.arpu,
        RANK() OVER (PARTITION BY d.city ORDER BY c.num_complaints_12m DESC) as complaint_rank
    FROM demographics d
    JOIN crm_data c ON d.customer_id = c.customer_id
    JOIN usage_billing u ON d.customer_id = u.customer_id
    WHERE c.num_complaints_12m > 0
)
SELECT *
FROM CityRankings
WHERE complaint_rank <= 3
ORDER BY city, complaint_rank;
"""

df_2 = run_query(query_2)
print(f"Identified {len(df_2)} high-priority customers for support outreach.")
display(df_2.head(10))

## 3. Creating a Customer 360 View
**Business Question:** Create a unified view combining demographics, usage, and predicted risk segmentation.
**SQL Technique:** `CASE WHEN`, Complex `JOIN`

In [ ]:
query_3 = """
SELECT 
    d.customer_id,
    d.age,
    u.plan_type,
    u.arpu,
    c.satisfaction_score,
    CASE 
        WHEN c.satisfaction_score >= 8 AND u.arpu >= 75 THEN 'Champion'
        WHEN c.satisfaction_score < 5 THEN 'High Risk'
        WHEN u.arpu > 90 THEN 'Whale'
        ELSE 'Standard'
    END as customer_segment
FROM demographics d
JOIN usage_billing u ON d.customer_id = u.customer_id
JOIN crm_data c ON d.customer_id = c.customer_id
LIMIT 10;
"""

df_3 = run_query(query_3)
display(df_3)

In [ ]:
conn.close()
print("Database connection closed. SQL Analysis Complete.")